# Preprocessing

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from tqdm import tqdm

FEATURES = ['volt1', 'volt2', 'amp1', 'amp2', 'FQtyL', 'FQtyR', 'E1 FFlow',
            'E1 OilT', 'E1 OilP', 'E1 RPM', 'E1 CHT1', 'E1 CHT2', 'E1 CHT3',
            'E1 CHT4', 'E1 EGT1', 'E1 EGT2', 'E1 EGT3', 'E1 EGT4', 'OAT', 'IAS',
            'VSpd', 'NormAc', 'AltMSL']



SHAPE = (4096, 23)  # Example shape, adjust as needed


def get_dataset(df):
    """
    Processes a DataFrame to create a PyTorch dataset.
    Args:
        df (pandas.DataFrame): The input DataFrame containing sensor data. 
                               It must have columns 'id' and 'before_after'.
    Returns:
        torch.utils.data.TensorDataset: A PyTorch dataset containing the processed sensor data and labels.
    The function performs the following steps:
    1. Extracts unique IDs from the DataFrame.
    2. For each unique ID, extracts the last SHAPE[0] rows of sensor data (first 23 columns).
    3. Pads the sensor data if it has fewer than SHAPE[0] rows.
    4. Converts the sensor data to a PyTorch tensor.
    5. Extracts the 'before_after' label for each ID.
    6. Stacks all sensor data tensors and labels into a single PyTorch dataset.
    """
    ids = df.id.unique()

    sensor_datas = []
    afters = []

    for id in tqdm(ids):
        sensor_data = df[df.id == id].iloc[-SHAPE[0]:, :23].values
        
        sensor_data = np.pad(sensor_data, ((0, SHAPE[0] - len(sensor_data)), (0, 0)))

        # Convert to PyTorch tensor
        sensor_data = torch.tensor(sensor_data, dtype=torch.float32)

        after = df[df.id == id]['before_after'].iloc[0]

        sensor_datas.append(sensor_data)
        afters.append(after)

    # Stack the tensors


    sensor_datas = torch.stack(sensor_datas)


    afters = torch.tensor(afters, dtype=torch.float32)

    # Create a PyTorch dataset
    dataset = torch.utils.data.TensorDataset(sensor_datas, afters)

    return dataset

def fix_type(x, y):
    return x.float(), y.float()

def prepare_for_training(ds, shuffle=False, repeat=False, predict=True, aug=False, batch_size=32, model_type='short_lstm', model_loss_type='bce'):
    """
    Prepares the dataset for training by applying various transformations such as shuffling, repeating, and batching.

    Args:
        ds (Dataset): The dataset to be prepared.
        shuffle (bool, optional): If True, shuffles the dataset. Defaults to False.
        repeat (bool, optional): If True, repeats the dataset. Defaults to False.
        predict (bool, optional): If True, prepares the dataset for prediction. Defaults to True.
        aug (bool, optional): If True, applies data augmentation. Defaults to False.
        batch_size (int, optional): The size of the batches. Defaults to 32.
        model_type (str, optional): The type of model being used. Defaults to 'short_lstm'.
        model_loss_type (str, optional): The type of loss function being used. Defaults to 'bce'.

    Returns:
        DataLoader: The prepared dataset wrapped in a DataLoader.
    """
    if shuffle:
        ds = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=True)
    else:
        ds = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=False, drop_last=True)

    if repeat:
        ds = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=shuffle, drop_last=True)

    if not predict:
        ds = [(x, x) for x, y in ds]
    else:
        if model_loss_type == 'bce':
            ds = [(x, y.view(-1, 1)) for x, y in ds]

    return ds


def get_train_and_val_for_fold(folded_datasets, fold, MODEL_LOSS_TYPE, NFOLD, AUGMENT, PREDICT):
    """
    Splits the provided datasets into training and validation sets for a given fold and prepares them for training.

    Args:
        folded_datasets (list): A list of datasets, where each dataset corresponds to a fold.
        fold (int): The index of the fold to be used as the validation set.
        MODEL_LOSS_TYPE (str): The type of model loss ('bce' for binary cross-entropy, 'mse' for mean squared error).
        NFOLD (int): The total number of folds.
        AUGMENT (bool): Whether to apply data augmentation to the training set.
        PREDICT (bool): Whether the datasets are being prepared for prediction.

    Returns:
        tuple: A tuple containing:
            - train_ds: The training dataset prepared for training.
            - val_ds: The validation dataset prepared for training.
            - mse_val_ds: The validation dataset prepared for training if MODEL_LOSS_TYPE is 'mse', otherwise None.
    """

    predict = True


    if MODEL_LOSS_TYPE == 'bce':
        train = []
        for i in range(NFOLD):
            if i == fold:
                val_ds = folded_datasets[i]
            else:
                train.append(folded_datasets[i])
    elif MODEL_LOSS_TYPE == 'mse':
        train = []
        for i in range(NFOLD):
            if i == fold:
                val_ds = folded_datasets[i][0].concatenate(folded_datasets[i][1])
            else:
                train.append(folded_datasets[i][0])

    train_ds = None
    for ds in train:
        train_ds = ds if train_ds is None else train_ds.concatenate(ds)

    mse_val_ds = None if not MODEL_LOSS_TYPE == 'mse' else prepare_for_training(val_ds, shuffle=False,  predict=True)
    train_ds = prepare_for_training(train_ds, shuffle=True, repeat = True, predict=PREDICT, aug = AUGMENT)
    val_ds = prepare_for_training(val_ds, shuffle=False,  predict=PREDICT)

    return train_ds, val_ds, mse_val_ds

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import torch

INPUT_COLUMNS = ['volt1',
 'volt2',
 'amp1',
 'amp2',
 'FQtyL',
 'FQtyR',
 'E1 FFlow',
 'E1 OilT',
 'E1 OilP',
 'E1 RPM',
 'E1 CHT1',
 'E1 CHT2',
 'E1 CHT3',
 'E1 CHT4',
 'E1 EGT1',
 'E1 EGT2',
 'E1 EGT3',
 'E1 EGT4',
 'OAT',
 'IAS',
 'VSpd',
 'NormAc',
 'AltMSL']



class DataLoading:
    def __init__(self, data_path):
        self.data_path = data_path
        self.df = None # original dataframe

    def load_data(self):
        df_test = pd.read_csv(self.data_path, nrows=100)

        float_cols = [c for c in df_test if df_test[c].dtype == "float64"]
        # Change float32_cols to use float32 instead of float16
        float32_cols = {c: np.float32 for c in float_cols}

        df = pd.read_csv(self.data_path, engine='c', dtype=float32_cols)
        df['id'] = df.id.astype('int32')
        self.df = df.dropna() # you can handle nans differently, but ymmv
        print(df.head(5))

        return df
    
    def min_max_scaling(self, input_columns, df=None):
        df = self.df if df is None else df
        qt = preprocessing.MinMaxScaler()
        try:
            qt.fit(df.loc[:, input_columns].sample(100000, random_state=0))
        except ValueError as e:
            if "could not convert string to float" in str(e):
                problematic_string = str(e).split(": ")[1].strip("'")
                print(f"Error: Could not convert string to float: '{problematic_string}'")

                for col in input_columns:
                    if problematic_string in df[col].astype(str).values:
                        print(f"Found problematic string in column: '{col}'")

            raise  # Re-raise the original exception after printing the information

        arr = df.loc[:, input_columns].values
        res = qt.transform(arr)

        for i, col in tqdm(enumerate(input_columns)):
            df.loc[:, col] = res[:, i]

        return df
    
    def get_folded_datasets(self,MODEL_LOSS_TYPE,df,NFOLD):
        folded_datasets = []

        for i in range(NFOLD):
            if MODEL_LOSS_TYPE == 'bce':
                folded_datasets.append(get_dataset(df[df.split == i]))
            elif MODEL_LOSS_TYPE == 'mse':
                after = get_dataset(df[(df.split == i) & (df.before_after == 1)])
                before = get_dataset(df[(df.split == i) & (df.before_after == 0)])
                folded_datasets.append((after, before))

        return folded_datasets
    

    def get_train_and_val_for_fold(self,folded_datasets,fold,MODEL_LOSS_TYPE='bce',NFOLD=5,AUGMENT=None, PREDICT=False): 
        
        if MODEL_LOSS_TYPE == 'bce':
            train = []
            for i in range(NFOLD):
                if i == fold:
                    val_ds = folded_datasets[i]
                else:
                    train.append(folded_datasets[i])
        elif MODEL_LOSS_TYPE == 'mse':
            train = []
            for i in range(NFOLD):
                if i == fold:
                    val_ds = folded_datasets[i][0].concatenate(folded_datasets[i][1])
                else:
                    train.append(folded_datasets[i][0])

        train_ds = None
        for ds in train:
            if isinstance(ds, torch.utils.data.TensorDataset):
                ds_tensors = ds.tensors
                for tensor in ds_tensors:
                    train_ds = tensor if train_ds is None else torch.cat((train_ds, tensor))
            else:
                train_ds = ds if train_ds is None else torch.cat((train_ds, ds))

        mse_val_ds = None if not MODEL_LOSS_TYPE == 'mse' else prepare_for_training(val_ds, shuffle=False,  predict=True)
        train_ds = prepare_for_training(train_ds, shuffle=True, repeat = True, predict=PREDICT, aug = AUGMENT)
        val_ds = prepare_for_training(val_ds, shuffle=False,  predict=PREDICT)

        return train_ds, val_ds, mse_val_ds

In [3]:
data_path = "/Users/manasdubey2022/Desktop/NGAFID/Codebase/data/NGAFID_MC_C37.csv"
data_loading = DataLoading(data_path)
data = data_loading.load_data()
data = data_loading.min_max_scaling(INPUT_COLUMNS)
print(data.head(5))

folded_datasets = data_loading.get_folded_datasets('bce',data,5)
print(type(folded_datasets[0]))
print(folded_datasets[0])


   volt1  volt2  amp1  amp2  FQtyL  FQtyR  E1 FFlow     E1 OilT    E1 OilP  \
0   27.9   27.9   7.9   0.7   24.0   24.0      2.09  129.199997  61.160000   
1   27.9   27.9   7.9   0.6   24.0   24.0      2.13  129.199997  61.200001   
2   27.9   28.0   8.0   0.6   24.0   24.0      2.07  129.199997  61.029999   
3   27.9   27.9   7.8   0.6   24.0   24.0      2.12  129.199997  61.160000   
4   27.9   27.9   7.7   0.6   24.0   24.0      2.08  129.100006  61.250000   

   E1 RPM  ...  OAT  IAS       VSpd  NormAc  AltMSL  id  plane_id  split  \
0  1191.0  ...  7.2  0.0  15.740000   -0.02   822.5   2        18      4   
1  1192.0  ...  7.2  0.0  11.130000   -0.00   822.5   2        18      4   
2  1186.0  ...  7.2  0.0  -0.850000    0.00     NaN   2        18      4   
3  1190.0  ...  7.2  0.0 -30.639999   -0.03   824.5   2        18      4   
4  1197.0  ...  7.2  0.0 -23.950001   -0.02   825.0   2        18      4   

   date_diff  before_after  
0         -2             1  
1         -2    

/Users/manasdubey2022/anaconda3/envs/ngafid_env/lib/python3.9/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
0it [00:00, ?it/s]/var/folders/k_/mxrh7nj55tl1jtzf2n_d0swm0000gn/T/ipykernel_4254/4237317180.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, col] = res[:, i]
23it [00:00, 27.46it/s]


      volt1     volt2      amp1      amp2  FQtyL  FQtyR  E1 FFlow   E1 OilT  \
0  0.830769  0.828125  0.481117  0.600000    1.0    1.0  0.124405  0.567582   
1  0.830769  0.828125  0.481117  0.577778    1.0    1.0  0.126786  0.567582   
3  0.830769  0.828125  0.479475  0.577778    1.0    1.0  0.126190  0.567582   
4  0.830769  0.828125  0.477832  0.577778    1.0    1.0  0.123810  0.567054   
5  0.830769  0.828125  0.476190  0.577778    1.0    1.0  0.122619  0.567054   

    E1 OilP    E1 RPM  ...       OAT       IAS      VSpd    NormAc    AltMSL  \
0  0.690678  0.419958  ...  0.569106  0.006946  0.582480  0.316832  0.030799   
1  0.691124  0.420310  ...  0.569106  0.006946  0.582047  0.326733  0.030799   
3  0.690678  0.419605  ...  0.569106  0.006946  0.578127  0.311881  0.030942   
4  0.691682  0.422073  ...  0.569106  0.006946  0.578754  0.316832  0.030978   
5  0.690232  0.419605  ...  0.566396  0.006946  0.580351  0.321782  0.030942   

   id  plane_id  split  date_diff  before_af

100%|██████████| 437/437 [00:01<00:00, 385.19it/s]


<class 'torch.utils.data.dataset.TensorDataset'>


In [4]:

from fastprogress import progress_bar

# Modify create_rocket_features to ensure correct tensor type and device
def create_rocket_features_max(dl, model, device='cpu'):
    model.eval()
    _x_out = []
    with torch.no_grad():
        for xb in dl:
            xb = xb.to(device).float()  # Ensure the input tensor is of the correct type and device
            features = model(xb).cpu()
            max_features = features[:, :model.n_kernels]  # Extract only the _max values
            _x_out.append(max_features)
    return torch.cat(_x_out)

In [5]:
from tsai.data.all import *
from tsai.all import ROCKET
import os

# Set the environment variable for MPS fallback
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_SIZE = 32 
NUM_KERNELS = 10000
KSS = [7, 9, 11]
CHANNLES_IN=23
SEQUENCE_LENGTH=4096    

# Create a ROCKET model
rocket = ROCKET(c_in=CHANNLES_IN, seq_len=SEQUENCE_LENGTH, n_kernels=NUM_KERNELS, kss=KSS).to(device)


#   Testing out on one fold

In [95]:
one_fold = folded_datasets[0]
print(f"Number of samples: {len(one_fold)}")
print(f"Shape of each sample: {one_fold.tensors[0].shape}")

Number of samples: 542
Shape of each sample: torch.Size([542, 4096, 23])


In [96]:
xb = torch.randn((32, 23, 4096))
print(len(xb[0]))

23


In [97]:
anomalies = []
normal_data = []

for data, label in one_fold:
    if label == 1:
        anomalies.append(data.permute(1, 0)) # change shape from (23, 4096) to (4096, 23)
    else:
        normal_data.append(data.permute(1, 0)) # change shape from (23, 4096) to (4096, 23)
        
#convert lists to Tensor datasets
anomalies = torch.stack(anomalies)
normal_data = torch.stack(normal_data)

print(anomalies.shape)
print(normal_data.shape)

#Create data loaders
anomalies_loader = torch.utils.data.DataLoader(anomalies, batch_size=32, shuffle=True)
normal_data_loader = torch.utils.data.DataLoader(normal_data, batch_size=32, shuffle=True)



torch.Size([224, 23, 4096])
torch.Size([318, 23, 4096])


In [98]:
print(f"Number of batches: {len(anomalies_loader)}")
for batch in anomalies_loader:
	print(f"Shape of each batch: {batch.shape}")
	break  # Print the shape of the first batch and break

Number of batches: 7
Shape of each batch: torch.Size([32, 23, 4096])


In [ ]:

# Extract features using the ROCKET model
anomalous_features = create_rocket_features_max(anomalies_loader, rocket, device=device)
normal_features = create_rocket_features_max(normal_data_loader, rocket, device=device)

In [68]:
print(normal_features.shape)

torch.Size([318, 10000])


In [ ]:
from sklearn.svm import OneClassSVM
import numpy as np
import matplotlib.pyplot as plt 

#Convert features to numpy arrays
anomalous_features = anomalous_features.numpy()
normal_features = normal_features.numpy()

# Fit the OC-SVM model on normal features
svm = OneClassSVM(kernel='rbf', gamma=0.001, nu=0.02)
svm.fit(normal_features)

# Predict anomalies in the anomalous features
pred = svm.predict(anomalous_features)
scores = svm.score_samples(anomalous_features)

# Calculate the threshold value for anomalies
thresh = np.quantile(scores, 0.03)
print("Threshold for anomalies:", thresh)

# Extract the anomalies
index = np.where(scores <= thresh)
anomalies = anomalous_features[index]

# Testing the SVM model on normal and anomalous features
test_normal_pred = svm.predict(normal_features)
test_anomalous_pred = svm.predict(anomalous_features)

# Calculate accuracy for normal and anomalous data
normal_accuracy = np.mean(test_normal_pred == 1)
anomalous_accuracy = np.mean(test_anomalous_pred == -1)

print(f"Accuracy on normal data: {normal_accuracy * 100:.2f}%")
print(f"Accuracy on anomalous data: {anomalous_accuracy * 100:.2f}%")


# All folds

In [6]:
anomalies = [] # post maintenance (0)
normal_data = [] # pre maintenance (1)
for fold in folded_datasets:
    for data, label in fold:
        if label == 1:
            normal_data.append(data.permute(1, 0)) # change shape from (23, 4096) to (4096, 23)
        else:
            anomalies.append(data.permute(1, 0)) # change shape from (23, 4096) to (4096, 23)

anomlies_dataset = torch.stack(anomalies)
normal_dataset = torch.stack(normal_data)

anomalies_loader = torch.utils.data.DataLoader(anomlies_dataset, batch_size=32, shuffle=True)
normal_loader = torch.utils.data.DataLoader(normal_dataset, batch_size=32, shuffle=True)


In [7]:
#Reveal properties of the DataLoaders so i can make tests in the future
print(f"Number of batches: {len(anomalies_loader)}")
for batch in anomalies_loader:
	print(f"Shape of each batch: {batch.shape}")
	break  # Print the shape of the first batch and break
    

Number of batches: 45
Shape of each batch: torch.Size([32, 23, 4096])


In [110]:
anomalous_features = create_rocket_features_max(anomalies_loader, rocket, device=device)
normal_features = create_rocket_features_max(normal_loader, rocket, device=device)

In [111]:
print(anomalous_features.shape)
print(normal_features.shape)

torch.Size([1413, 10000])
torch.Size([976, 10000])


In [112]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Define the number of folds and repetitions

k = 5
repetitions = 10

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'nu': [0.01, 0.05, 0.1, 0.2, 0.5]
}

# Define the custom scoring function for GridSearchCV
def custom_f1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, pos_label=-1)

scorer = make_scorer(custom_f1_score, greater_is_better=True)

grid_search = GridSearchCV(OneClassSVM(), param_grid, scoring=scorer, cv=5, n_jobs=-1)

#Fit the grid search on normal features
grid_search.fit(normal_features)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)


kf = KFold(n_splits=k, shuffle=True, random_state=42)


f1_scores = []

In [ ]:
anomalous_features = anomalous_features.numpy()
normal_features = normal_features.numpy()

# OC-SVM

In [117]:
from sklearn.metrics import f1_score



for train_index, test_index in kf.split(normal_features):
    for _ in range(repetitions):
        # Split the normal features into training and validation sets
        X_train, X_test_normal = normal_features[train_index], normal_features[test_index]

        # Combine the validation set with the anomalous features to form the test set
        X_test = np.vstack((X_test_normal, anomalous_features))
        y_test = np.hstack((np.ones(len(X_test_normal)), -np.ones(len(anomalous_features))))

        # Fit the OC-SVM model on the training set
        svm = OneClassSVM(kernel=best_params['kernel'], gamma=best_params['gamma'], nu=best_params['nu']) 

        # Calculate the anomaly scores for the training data
        train_scores = svm.score_samples(X_train)

        # Compute the mean and standard deviation of the anomaly scores
        mean_train_score = np.mean(train_scores)
        std_train_score = np.std(train_scores)

        # Calculate the threshold using the formula
        tau = mean_train_score - 3 * std_train_score

        # Calculate the anomaly scores for the test data
        test_scores = svm.score_samples(X_test)

        # Predict anomalies using the threshold
        y_pred = np.where(test_scores <= tau, -1, 1)

        # Calculate the F1-score
        f1 = f1_score(y_test, y_pred, pos_label=-1)
        f1_scores.append(f1)

# Calculate the average F1-score and standard deviation
avg_f1_score = np.mean(f1_scores)
std_f1_score = np.std(f1_scores)

print(f"Average F1-score: {avg_f1_score:.4f}")
print(f"Standard deviation of F1-score: {std_f1_score:.4f}")

Average F1-score: 0.9340
Standard deviation of F1-score: 0.0003


# Isol